## Function Calling. Calling Multiple Functions.

The latest versions of gpt-35-turbo and gpt-4 have been fine-tuned to work with functions and are able to both determine when and how a function should be called.

If one or more functions are included in your request, the model will then determine if any of the functions should be called based on the context of the prompt.

When the model determines that a function should be called, it will then respond with a JSON object including the arguments for the function.

https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/function-calling

In [9]:
import openai
import json
import os
from IPython.display import display, HTML, JSON, Markdown
from dotenv import load_dotenv

In [10]:
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")

# Configure OpenAI API
openai.api_type = "azure"
# The API version required for Function Calling
openai.api_version = "2023-07-01-preview"
openai.api_base = OPENAI_DEPLOYMENT_ENDPOINT
openai.api_key = OPENAI_API_KEY

### Calling multiple functions

#### Define functions

In [11]:
import pytz
from datetime import datetime


def get_current_time(location):
    try:
        # Get the timezone for the city
        timezone = pytz.timezone(location)

        # Get the current time in the timezone
        now = datetime.now(timezone)
        current_time = now.strftime("%I:%M:%S %p")

        return current_time
    except:
        return "Sorry, I couldn't find the timezone for that location."

In [12]:
get_current_time("America/New_York")

'08:19:57 AM'

In [13]:
import math


def calculator(num1, num2, operator):
    if operator == '+':
        return str(num1 + num2)
    elif operator == '-':
        return str(num1 - num2)
    elif operator == '*':
        return str(num1 * num2)
    elif operator == '/':
        return str(num1 / num2)
    elif operator == '**':
        return str(num1 ** num2)
    elif operator == 'sqrt':
        return str(math.sqrt(num1))
    else:
        return "Invalid operator"

In [14]:
print(calculator(5, 5, '+'))

10


#### Define functions prompt

In [15]:
functions = [
    {
        "name": "get_current_time",
        "description": "Get the current time in a given location",
        "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The location name. The pytz is used to get the timezone for that location. Location names should be in a format like America/New_York, Asia/Bangkok, Europe/London",
                    }
                },
            "required": ["location"],
        },
    },
    {
        "name": "calculator",
        "description": "A simple calculator used to perform basic arithmetic operations",
        "parameters": {
                "type": "object",
                "properties": {
                    "num1": {"type": "number"},
                    "num2": {"type": "number"},
                    "operator": {"type": "string", "enum": ["+", "-", "*", "/", "**", "sqrt"]},
                },
            "required": ["num1", "num2", "operator"],
        },
    }
]

available_functions = {
    "get_current_time": get_current_time,
    "calculator": calculator,
}

In [16]:
import inspect

# helper method used to run validation on the function arguments returned by the model

def check_args(function, args):
    sig = inspect.signature(function)
    print (f""" function signature {sig}\n""")
    params = sig.parameters
    print (f""" function parameters {params}\n""")

    # Check if there are extra arguments
    for name in args:
        if name not in params:
            return False
    # Check if the required arguments are provided
    for name, param in params.items():
        if param.default is param.empty and name not in args:
            return False

    return True

#### Orchestrating function calls

In [17]:
def run_conversation(messages, functions, available_functions, deployment_id):
    
    # Step 1: send the conversation and available functions to GPT

    response = openai.ChatCompletion.create(
        deployment_id=deployment_id,
        messages=messages,
        functions=functions,
        function_call="auto",
    )
    response_message = response["choices"][0]["message"]

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        print("Recommended Function call:")
        print(response_message.get("function_call"))
        print()

        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors

        function_name = response_message["function_call"]["name"]

        # verify function exists
        if function_name not in available_functions:
            return "Function " + function_name + " does not exist"
        function_to_call = available_functions[function_name]

        # verify function has correct number of arguments
        function_args = json.loads(
            response_message["function_call"]["arguments"])
        if check_args(function_to_call, function_args) is False:
            return "Invalid number of arguments for function: " + function_name
        
        #call the function 
        function_response = function_to_call(**function_args)

        print("Output of function call:")
        print(function_response)
        print()

        # Step 4: send the info on the first function call and function response to GPT

        # adding assistant response to messages
        messages.append(
            {
                "role": response_message["role"],
                "name": response_message["function_call"]["name"],
                "content": response_message["function_call"]["arguments"],
            }
        )

        # adding function response to messages
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response

        print("Messages in second request:")
        for message in messages:
            print(message)
        print()

        second_response = openai.ChatCompletion.create(
            messages=messages,
            deployment_id=deployment_id
        )  # get a new response from GPT where it can see the function response

        return second_response

In [18]:
messages = [{"role": "user", "content": "What time is it in New York?"}]
assistant_response = run_conversation(
    messages, functions, available_functions, "gpt-35-turbo-0613")
print(assistant_response['choices'][0]['message'])

Recommended Function call:
{
  "name": "get_current_time",
  "arguments": "{\n  \"location\": \"America/New_York\"\n}"
}

 function signature (location)

 function parameters OrderedDict([('location', <Parameter "location">)])

Output of function call:
08:26:11 AM

Messages in second request:
{'role': 'user', 'content': 'What time is it in New York?'}
{'role': 'assistant', 'name': 'get_current_time', 'content': '{\n  "location": "America/New_York"\n}'}
{'role': 'function', 'name': 'get_current_time', 'content': '08:26:11 AM'}

{
  "role": "assistant",
  "content": "It is currently 08:26 AM in New York."
}


In [19]:
messages = [
    {"role": "user", "content": "Last month Fabrikam made $73,846 in sales. Based on that, what would the annual run rate be?"}
]

assistant_response = run_conversation(
    messages, functions, available_functions, "gpt-35-turbo-0613")
print(assistant_response['choices'][0]['message'])

Recommended Function call:
{
  "name": "calculator",
  "arguments": "{\n  \"num1\": 73846,\n  \"num2\": 12,\n  \"operator\": \"*\"\n}"
}

 function signature (num1, num2, operator)

 function parameters OrderedDict([('num1', <Parameter "num1">), ('num2', <Parameter "num2">), ('operator', <Parameter "operator">)])

Output of function call:
886152

Messages in second request:
{'role': 'user', 'content': 'Last month Fabrikam made $73,846 in sales. Based on that, what would the annual run rate be?'}
{'role': 'assistant', 'name': 'calculator', 'content': '{\n  "num1": 73846,\n  "num2": 12,\n  "operator": "*"\n}'}
{'role': 'function', 'name': 'calculator', 'content': '886152'}

{
  "role": "assistant",
  "content": "The annual run rate would be $886,152."
}
